In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import least_squares
import yfinance as yf

# Download data

In [2]:
load_from = pd.to_datetime('1995-01-01')  # Need at least 4 years prior to the first training day
train_start_date = pd.to_datetime('2000-01-01')
test_start_date = pd.to_datetime('2019-01-01')
test_end_date = pd.to_datetime('2022-05-15')

In [3]:
# # Load data on SPX and VIX
# spx_data = yf.Ticker("^GSPC").history(start=load_from, end=test_end_date)
# vix_data = yf.Ticker("^VIX").history(start=load_from, end=test_end_date)

In [4]:
# spx_data.to_csv('spx_data.csv')
# vix_data.to_csv('vix_data.csv')

In [5]:
spx_data = pd.read_csv('spx_data.csv', index_col=0, parse_dates=[0])
vix_data = pd.read_csv('vix_data.csv', index_col=0, parse_dates=[0])

In [6]:
spx_data.index = pd.to_datetime(spx_data.index.date)
vix_data.index = pd.to_datetime(vix_data.index.date)

In [7]:
# create new df that calculates the daily returns and squared daily returns of SPX
spx = pd.DataFrame(columns=['r1', 'r2'])
spx['r1'] = spx_data.loc[train_start_date:test_start_date-pd.Timedelta(days=1), 'Close'].pct_change()
spx['r2'] = spx['r1'] ** 2
spx

,r1,r2
2000-01-03,NaN,NaN
2000-01-04,-0.038345,1.470314e-03
2000-01-05,0.001922,3.694787e-06
2000-01-06,0.000956,9.133206e-07
2000-01-07,0.027090,7.338897e-04
...,...,...
2018-12-24,-0.027112,7.350743e-04
2018-12-26,0.049594,2.459539e-03
2018-12-27,0.008563,7.331950e-05
2018-12-28,-0.001242,1.541527e-06


In [8]:
vix = vix_data.loc[train_start_date:test_start_date-pd.Timedelta(days=1), ['Close']] / 100
vix.columns = ['vix']
vix

,vix
2000-01-03,0.2421
2000-01-04,0.2701
2000-01-05,0.2641
2000-01-06,0.2573
2000-01-07,0.2172
...,...
2018-12-24,0.3607
2018-12-26,0.3041
2018-12-27,0.2996
2018-12-28,0.2834


In [9]:
data = pd.concat([spx, vix], axis=1)
data = data.dropna()
data

,r1,r2,vix
2000-01-04,-0.038345,1.470314e-03,0.2701
2000-01-05,0.001922,3.694787e-06,0.2641
2000-01-06,0.000956,9.133206e-07,0.2573
2000-01-07,0.027090,7.338897e-04,0.2172
2000-01-10,0.011190,1.252154e-04,0.2171
...,...,...,...
2018-12-24,-0.027112,7.350743e-04,0.3607
2018-12-26,0.049594,2.459539e-03,0.3041
2018-12-27,0.008563,7.331950e-05,0.2996
2018-12-28,-0.001242,1.541527e-06,0.2834


# High clarity implementation (low performance)

In [10]:
def exp_kernel(t, λ, θ):
    '''
    Exponential kernel function
    '''
    return θ * λ * np.exp(-λ * t)

def convex_combi_exp_kernel(t, λ1, λ2, θ):
    '''
    Convex combination of two exponential kernel functions
    '''
    return exp_kernel(t, λ1, 1-θ) + exp_kernel(t, λ2, θ)

def R1(λ1, λ2, θ, df):
    '''
    Calculate R1 which is the simple return weighted by convex combination of two exponential kernel functions
    '''
    dt = (df.index[-1] - df.index).days / 365
    return np.sum(df['r1'].values * convex_combi_exp_kernel(dt, λ1, λ2, θ))

def R2(λ1, λ2, θ, df):
    '''
    Calculate R2 which is the squared return weighted by convex combination of two exponential kernel functions
    '''
    dt = (df.index[-1] - df.index).days / 365
    return np.sqrt(np.sum(df['r2'].values * convex_combi_exp_kernel(dt, λ1, λ2, θ)))

def predict(β0, β1, β2, λ11, λ12, θ1, λ21, λ22, θ2, df):
    '''
    Predict the VIX level using the PDV model
    '''
    return β0 + β1 * R1(λ11, λ12, θ1, df[['r1']]) + β2 * R2(λ21, λ22, θ2, df[['r2']])

def residual(x, df, n):
    '''
    Calculate the residuals of the PDV model
    '''
    β0, β1, β2, λ11, λ12, θ1, λ21, λ22, θ2 = x
    residuals = []
    for i in range(len(df) - n + 1):
        y = df.iloc[i + n - 1]['vix'] - predict(β0, β1, β2, λ11, λ12, θ1, λ21, λ22, θ2, df.iloc[i:i + n - 1])
        residuals.append(y)
    return np.array(residuals)

In [11]:
window = 1000
lower_bound = [-np.inf, -np.inf, -np.inf, 0., 0., 0., 0., 0., 0.]
upper_bound = [np.inf, np.inf, np.inf, np.inf, np.inf, 1., np.inf, np.inf, 1.]
res = least_squares(residual, [0.054, -0.078, 0.82, 52.8, 3.79, 0.81, 17.3, 1.16, 0.43], args=(data, window), bounds=(lower_bound, upper_bound), verbose=2, ftol=1e-6)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0252e+00                                    7.53e+00    
       1              3         9.9800e-01      2.72e-02       1.87e+00       1.57e+00    
       2              4         9.9095e-01      7.04e-03       1.65e+00       1.01e+00    
       3              5         9.8774e-01      3.22e-03       1.27e+01       2.22e+00    
       4              6         9.8255e-01      5.19e-03       2.44e+00       5.39e-01    
       5              7         9.8169e-01      8.59e-04       3.30e+00       7.29e-01    
       6              8         9.8113e-01      5.63e-04       4.96e+00       4.64e-02    
       7              9         9.8089e-01      2.34e-04       1.70e+00       1.45e-01    
       8             10         9.8085e-01      4.35e-05       6.60e+00       3.37e-01    
       9             11         9.8066e-01      1.85e-04       1.65e+00       2.04e-02    

In [12]:
res.keys()

dict_keys(['x', 'cost', 'fun', 'jac', 'grad', 'optimality', 'active_mask', 'nfev', 'njev', 'status', 'message', 'success'])

In [13]:
res.x

array([ 4.36694850e-02, -3.59595684e-02,  8.83069034e-01,  6.84085357e+01,
        2.07704064e+01,  8.94804840e-01,  1.28287906e+01,  1.09884865e+00,
        3.17378721e-01])

In [14]:
residuals = residual(res.x, data, window)
total_sum_of_squares = np.sum((data.iloc[window-1:]['vix'] - np.mean(data.iloc[window-1:]['vix'])) ** 2)
print('Mean: {:.4f}, Min: {:.4f}, Max: {:.4f}, MAE: {:.4f}, MSE: {:.4f}, R^2: {:.4f}'.format(
    np.mean(residuals), np.min(residuals), np.max(residuals), np.mean(np.abs(residuals)), np.mean(residuals**2), (1-np.sum(residuals**2)/total_sum_of_squares)))

Mean: -0.0000, Min: -0.1470, Max: 0.2217, MAE: 0.0156, MSE: 0.0005, R^2: 0.9330


# Vectorized implementation

In [15]:
# implement the residual function for the PDV model but with vectorized calculation instead of for loop

import torch # faster with torch than numpy

def exp_kernel_vec(t, λ, θ):
    '''
    Exponential kernel function torch version
    t is a tensor of shape (len(df) - n, n)
    '''
    return θ * λ * torch.exp(-λ * t)

def convex_combi_exp_kernel_vec(t, λ1, λ2, θ):
    '''
    Convex combination of two exponential kernel functions torch version
    t is a tensor of shape (len(df) - n, n)
    '''
    return exp_kernel_vec(t, λ1, 1-θ) + exp_kernel_vec(t, λ2, θ)

def R1_vec(λ1, λ2, θ, return_tensor, t):
    '''
    Calculate R1 which is the simple return weighted by convex combination of two exponential kernel functions torch version
    '''

    return torch.sum(return_tensor * convex_combi_exp_kernel_vec(t, λ1, λ2, θ), dim=1)

def R2_vec(λ1, λ2, θ, return_tensor, t):
    '''
    Calculate R2 which is the squared return weighted by convex combination of two exponential kernel functions torch version
    '''
    return torch.sqrt(torch.sum(return_tensor * convex_combi_exp_kernel_vec(t, λ1, λ2, θ), dim=1))

def predict_vec(β0, β1, β2, λ11, λ12, θ1, λ21, λ22, θ2, df, n):
    '''
    Predict the VIX level using the PDV model torch version
    '''
    array = df[['r1', 'r2', 'vix']].values

    # create sliding window of size n for r1 and r2 with shape (len(df) - n + 1, n)
    r1_sliding_window = torch.tensor(np.lib.stride_tricks.sliding_window_view(array[:, 0], n), dtype=torch.float64, requires_grad=False)
    r2_sliding_window = torch.tensor(np.lib.stride_tricks.sliding_window_view(array[:, 1], n), dtype=torch.float64, requires_grad=False)

    # calcualte time difference between prediction datetime and each row in the sliding window with shape (len(df) - n + 1, n)
    dt = ((df.index[1:] - df.index[:-1]).days / 365).values
    dt_sliding_window = np.lib.stride_tricks.sliding_window_view(dt, n-1)
    dt_sliding_window = np.flip(dt_sliding_window, axis=1)
    dt_sliding_window = np.cumsum(dt_sliding_window, axis=1)
    dt_sliding_window = np.flip(dt_sliding_window, axis=1)
    dt_sliding_window = np.concatenate((dt_sliding_window, np.zeros((dt_sliding_window.shape[0], 1))), axis=1)
    t = torch.tensor(dt_sliding_window, dtype=torch.float64, requires_grad=False)

    # print(r1_sliding_window.shape, r2_sliding_window.shape, t.shape)

    return β0 + β1 * R1_vec(λ11, λ12, θ1, r1_sliding_window, t) + β2 * R2_vec(λ21, λ22, θ2, r2_sliding_window, t)

def residual_vec(x, df, n):
    '''
    Calculate the residuals of the PDV model torch version
    '''
    β0, β1, β2, λ11, λ12, θ1, λ21, λ22, θ2 = x
    y = df.iloc[n-1:]['vix'].values # shape (len(df) - n + 1,) = number of predictions
    y_hat = predict_vec(β0, β1, β2, λ11, λ12, θ1, λ21, λ22, θ2, df, n)
    return y - y_hat.numpy()

In [16]:
window = 1000
lower_bound = [-np.inf, -np.inf, -np.inf, 0., 0., 0., 0., 0., 0.]
upper_bound = [np.inf, np.inf, np.inf, np.inf, np.inf, 1., np.inf, np.inf, 1.]
res = least_squares(residual_vec, [0.054, -0.078, 0.82, 52.8, 3.79, 0.81, 17.3, 1.16, 0.43], args=(data, window), bounds=(lower_bound, upper_bound), verbose=2, ftol=1e-6)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         6.6286e-01                                    7.41e+00    
       1              3         6.3430e-01      2.86e-02       1.42e+00       2.99e+00    
       2              4         6.2139e-01      1.29e-02       1.42e+00       2.18e-01    
       3              5         6.2010e-01      1.29e-03       9.51e+00       2.66e+00    
       4              6         6.1401e-01      6.09e-03       2.85e+00       4.90e-01    
       5              7         6.1313e-01      8.83e-04       5.69e+00       4.23e-01    
       6              9         6.1275e-01      3.78e-04       2.85e+00       8.42e-02    
       7             10         6.1261e-01      1.34e-04       5.69e+00       2.06e-01    
       8             11         6.1250e-01      1.10e-04       5.69e+00       1.36e-01    
       9             12         6.1248e-01      2.54e-05       4.90e+00       7.72e-02    

In [17]:
res.x

array([ 4.35885953e-02, -4.12544816e-02,  8.85362887e-01,  6.55315948e+01,
        1.83999483e+01,  8.37978808e-01,  1.25555836e+01,  1.11373321e+00,
        3.26484201e-01])

In [18]:
lower_bound = [-np.inf, -np.inf, -np.inf, 0., 0., 0., 0., 0., 0.]
upper_bound = [np.inf, np.inf, np.inf, np.inf, np.inf, 1., np.inf, np.inf, 1.]
res = least_squares(residual_vec, [0.1, -1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.5], args=(data, window), bounds=(lower_bound, upper_bound), verbose=2, ftol=1e-6)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.3367e+01                                    2.60e+01    
       1              2         4.2952e+00      9.07e+00       7.21e-01       1.36e+01    
       2              4         2.8369e+00      1.46e+00       3.52e-01       2.40e+00    
       3              5         2.4008e+00      4.36e-01       7.04e-01       5.14e+00    
       4              6         1.9307e+00      4.70e-01       7.04e-01       1.87e+00    
       5              7         1.6661e+00      2.65e-01       1.41e+00       5.65e+00    
       6              8         1.3830e+00      2.83e-01       1.41e+00       2.24e+00    
       7              9         1.2541e+00      1.29e-01       2.81e+00       7.11e+00    
       8             10         1.0859e+00      1.68e-01       2.81e+00       2.42e+00    
       9             11         9.4190e-01      1.44e-01       5.13e+00       7.84e+00    

In [19]:
res.x

array([ 4.35949667e-02, -4.12812011e-02,  8.85331051e-01,  1.83282080e+01,
        6.50128185e+01,  1.64754191e-01,  1.25581726e+01,  1.11401299e+00,
        3.26593119e-01])

In [20]:
residuals = residual_vec(res.x, data, window)
total_sum_of_squares = np.sum((data.iloc[window-1:]['vix'] - np.mean(data.iloc[window-1:]['vix'])) ** 2)
print('Mean: {:.4f}, Min: {:.4f}, Max: {:.4f}, MAE: {:.4f}, MSE: {:.4f}, R^2: {:.4f}'.format(
    np.mean(residuals), np.min(residuals), np.max(residuals), np.mean(np.abs(residuals)), np.mean(residuals**2), (1-np.sum(residuals**2)/total_sum_of_squares)))

Mean: 0.0000, Min: -0.1030, Max: 0.1349, MAE: 0.0129, MSE: 0.0003, R^2: 0.9582
